In [20]:
import numpy as np

def softmax(x):
    """
    Compute the softmax of vector x.
    so along axis 1
    """
    # normalize
    x = x - np.min(x, axis=1, keepdims=True)
    x = x/np.max(x, axis=1, keepdims=True)*16

    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

import numpy as np

# Assuming DMODEL, N, BITWIDTH, etc., are defined
DMODEL = 16  # Example values, adjust as necessary
N = 16       # Example values, adjust as necessary
BITWIDTH = 4 # Bit-width for data_t
BITWIDTH3 = 12 # Bit-width for data3_t

In [6]:
import re
import numpy as np


def read_weights(file_path):
    with open(file_path, "r") as file:
        content = file.read()

    # Regular expression to find matrices
    matrices = re.findall(r"{\{(.*?)\}\}", content, re.DOTALL)

    make_int = lambda x: int(x.strip().strip("{").strip("}"))

    # Process and store each matrix
    Q_W = np.array(
        [
            list(map(make_int, row.strip(",").split(",")))
            for row in matrices[0].split("\n")
            if row.strip() != ""
        ]
    )
    K_W = np.array(
        [
            list(map(make_int, row.strip(",").split(",")))
            for row in matrices[1].split("\n")
            if row.strip() != ""
        ]
    )
    V_W = np.array(
        [
            list(map(make_int, row.strip(",").split(",")))
            for row in matrices[2].split("\n")
            if row.strip() != ""
        ]
    )

    return Q_W, K_W, V_W


# Example usage
file_path = f"weights{DMODEL}.h"  # Replace with the actual file path
Q_W, K_W, V_W = read_weights(file_path)


In [15]:
def normalize(x):
    # normalize matrix along axis 1
    x = x - np.min(x, axis=1, keepdims=True)
    x = x/np.max(x, axis=1, keepdims=True)*16
    return x

In [19]:
import numpy as np

# Assuming DMODEL, N, Q_W, K_W, V_W are defined

# Step 1: Generate and normalize random tokens
tokens = np.random.rand(N, DMODEL) * 15

# Normalize Q, K, V matrices
Q_W_norm = Q_W #/ np.linalg.norm(Q_W, axis=1, keepdims=True)
K_W_norm = K_W #/ np.linalg.norm(K_W, axis=1, keepdims=True)
V_W_norm = V_W #/ np.linalg.norm(V_W, axis=1, keepdims=True)

# Step 2: Apply QKV transformations
Q = tokens @ Q_W_norm
K = tokens @ K_W_norm
V = tokens @ V_W_norm

print("Q")
print(Q)
print("K")
print(K)
print("V")
print(V)
print("--------------------")

K = normalize(K)
Q = normalize(Q)

# Step 3: Compute attention using softmax
K_T = K.T
attention_scores = Q @ K_T #/ np.sqrt(DMODEL)


print("attention scores")
print(attention_scores)

attention_weights = softmax(attention_scores)

print(attention_weights)

V = normalize(V)
attention_output = attention_weights @ V

print(attention_output.astype(int))

# Step 4: Normalize output tokens back into the range 0-15
#output_tokens = np.clip(np.round((attention_output - np.min(attention_output)) / (np.max(attention_output) - np.min(attention_output)) * 15), 0, 15).astype(int)

# Output tokens
#print(output_tokens)


Q
[[ 801.53012722  946.93973881  913.5587464  1124.59585397 1051.59173778
   805.64256149 1088.78519547  942.36564434 1220.19795065  783.48112208
   876.98187764 1030.11858235 1148.95249895  847.90122709  587.84066046
  1005.29113441]
 [ 920.30221018 1116.19623532 1129.99537878 1238.81904556 1012.21539924
  1065.5863008  1317.45502669 1027.97278475 1439.75708096  965.19156908
  1137.19194349 1083.24159601 1327.6330733   917.94940723  737.1243218
  1187.92215395]
 [ 704.59398352  806.42140586  782.94452195  896.23654769  753.01746529
   823.04013187  876.38990625  735.15732584  931.30379824  639.75669345
   884.51614335  737.76026784 1027.71149429  855.25008871  566.17231603
   921.76569368]
 [ 677.58161147  723.81748089  761.73448381  917.16116268  726.45165108
   747.9522485   820.08507434  593.27932557  840.94795751  673.80016974
   735.83529843  858.08788659  818.00288037  763.05901969  587.58324552
   788.08550905]
 [ 636.26585307  849.05665572  938.82500569  998.54831312  963.8355

In [30]:
def print_matrix(matrix):
    #print(f"{name} = np.array([")
    for row in matrix:
        for element in row:
            print(f"{element} ", end="")
        print()
    

In [37]:
# save 10 input output pairs to file
# as N*DMODEL

file = open("generate_tests_input.txt", "w")
file2 = open("generate_tests_output.txt", "w")

np.random.seed(0)

for i in range(10):
    tokens = np.random.rand(N, DMODEL) * 15
    Q = normalize(tokens) @ normalize(Q_W.T)
    K = normalize(tokens) @ normalize(K_W.T)
    V = normalize(tokens) @ normalize(V_W.T)

    K_T = K.T
    attention_scores = Q @ K_T #/ np.sqrt(DMODEL)

    attention_weights = softmax(attention_scores)

    V = normalize(V)
    attention_output = attention_weights @ V

    # print max in each row
    print("max index in each row")

    for j in range(len(attention_scores)):
        print(np.argmax(attention_scores[j]))

    print("--------------------")
    print_matrix(V.astype(int))
    print("--------------------")

    flat_tokens = tokens.flatten()
    flat_output = attention_output.flatten()

    for j in range(len(flat_tokens)):
        file.write(str(int(flat_tokens[j])) + "\n")
    #file.write("\n")

    for j in range(len(flat_output)):
        file2.write(str(int(flat_output[j])) + "\n")
    #file2.write("\n")

file.close()
file2.close()

max index in each row
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
--------------------
0 14 11 15 1 3 13 4 13 8 16 6 8 11 7 1 
0 16 7 4 6 3 9 1 12 4 12 8 8 8 3 2 
3 16 12 9 0 5 11 4 11 6 12 7 6 14 6 3 
0 12 13 8 9 12 11 7 16 8 6 9 12 5 5 7 
2 15 16 5 3 6 8 2 15 11 8 5 10 9 0 1 
1 12 4 8 0 8 11 1 16 3 7 3 4 5 2 2 
0 16 7 6 5 4 7 5 8 6 13 6 9 10 2 1 
0 16 11 9 1 2 8 1 12 7 12 6 6 9 4 3 
0 15 8 6 6 5 9 5 7 9 16 6 7 15 5 2 
0 16 9 9 2 4 12 2 11 5 12 9 6 8 8 4 
0 16 14 9 6 8 11 6 13 10 10 11 14 8 7 6 
0 10 13 8 5 3 11 3 14 7 11 5 9 16 5 0 
0 16 9 6 8 6 7 8 12 7 10 8 14 12 5 0 
0 15 10 10 11 5 12 8 11 6 16 9 11 13 7 4 
0 15 15 8 10 7 14 6 16 5 13 11 13 12 4 5 
0 14 12 8 7 9 13 8 16 7 13 9 9 13 8 8 
--------------------
max index in each row
1
7
7
7
7
1
7
1
7
1
7
1
1
7
7
7
--------------------
0 16 8 8 10 9 9 10 10 10 13 9 14 10 6 1 
0 16 9 5 2 5 8 3 10 4 8 8 10 10 3 0 
0 13 10 11 7 4 12 3 14 0 12 6 5 16 8 6 
0 12 10 5 7 9 8 10 10 3 10 7 8 16 4 6 
0 16 5 7 3 9 8 6 12 6 13 8 9 11 4 1 
0 8 9 10 7 12 14 10 